# Reporte de producción mensual

In [1]:
import streamlit as st
import mysql.connector
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
def format_timedelta_hh_mm_ss(td: timedelta) -> str:
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

config = {
  'user': 'operario',
  'password': '',
  'host': '192.168.0.12', # O la IP de tu servidor
  'database': 'dbp8100'
}

def conectar_mysql():
    return mysql.connector.connect(**config)
    
def obtener_df(fecha_inicio, fecha_fin):
    db = conectar_mysql()
    cur = db.cursor()
    cur.execute('''
                SELECT                 
                    tareaseje.NroID,
                    tareaseje.Fecha AS Fecha,
                    tareaseje.Hora AS Hora,
                    formulas.NroID AS IDF,
                    formulas.Nombre AS Nombre,
                    tareaseje.Set AS Programado,
                    (SELECT SUM(dcaptura.Valor) FROM dbp8100.dcaptura WHERE dcaptura.IDT = tareaseje.NroID) AS Dosificado,
                    tareaseje.Tiempo AS Tiempo
                FROM 
                    dbp8100.tareaseje AS tareaseje
                JOIN 
                    dbp8100.formulas AS formulas ON formulas.NroID = tareaseje.IDF
                WHERE
                    (tareaseje.Fecha = %s AND tareaseje.Hora >= '22:00:00') OR 
                    (tareaseje.Fecha > %s AND tareaseje.Fecha <= %s AND tareaseje.Hora < '22:00:00')
                GROUP BY
                    tareaseje.NroID
                ''',(fecha_inicio, fecha_inicio, fecha_fin))

    resultados = cur.fetchall()
    # Obtener los nombres de las columnas
    columnas = [desc[0] for desc in cur.description]

    # Crear un DataFrame de pandas
    df = pd.DataFrame(resultados, columns=columnas)

    return df

In [3]:
df = obtener_df('2025-08-07', '2025-08-08')

DatabaseError: 2005 (HY000): Unknown MySQL server host '200.107.105.35:33066' (-2)

In [19]:
df

,NroID,Fecha,Hora,IDF,Nombre,Programado,Dosificado,Tiempo
0,26693,2025-08-07,0 days 23:18:29,1209,CABAÑA PRESENTACIÓN (LA MALVINA),6000.0,6033.90,0 days 00:46:53
1,26694,2025-08-08,0 days 00:20:01,1240,(LA MALVINA) DESARROLLO TERNEROS MAIZ SIN SAL,6000.0,6018.10,0 days 00:51:48
2,26695,2025-08-08,0 days 01:15:36,1155,MANTENIMIENTO MELAZA....,8000.0,8093.85,0 days 01:00:50
3,26696,2025-08-08,0 days 02:20:01,1268,CREC. MELAZA 12 % A.A....,15000.0,15193.90,0 days 02:20:39
4,26697,2025-08-08,0 days 04:41:55,469,FIBROMIXER RECRIA 14% A.A.,12000.0,12094.10,0 days 01:47:06
5,26698,2025-08-08,0 days 06:30:53,369,DESTETE PRECOZ MELAZA 10% A.A. EXPELLER,12000.0,7086.70,0 days 01:26:32
6,26699,2025-08-08,0 days 07:58:20,369,DESTETE PRECOZ MELAZA 10% A.A. EXPELLER,5000.0,5111.40,0 days 00:36:28
7,26700,2025-08-08,0 days 08:35:06,773,FIBROMIXER PASTOREO 16%A.A. (P Girasol Común),7000.0,7063.60,0 days 01:30:03
8,26701,2025-08-08,0 days 10:06:06,1120,RECRIA 16 AL 15% A.A....,40000.0,1033.50,0 days 00:10:38
9,26702,2025-08-08,0 days 10:17:01,1120,RECRIA 16 AL 15% A.A....,39000.0,9054.60,0 days 00:00:00
